# Demo: Локальный запуск скриптов для предсказания пространственно-временных рядов

Ноутбук показывает, как подготовить данные, проверить интерфейсы локальных скриптов и запустить предсказание без поднятия FastAPI. Полезно для отладки и быстрого прототипирования.

Цель: запускать модули и CLI-скрипты локально, генерировать тестовый датасет и проверять выходные файлы/метрики.

## 1) Установка зависимостей и проверка окружения

Выполните в терминале следующую проверку и установку зависимостей, если это ещё не сделано:

```bash
python --version
pip install -r requirements.txt
```

Проверьте, что виртуальное окружение активно и переменные окружения (`HF_MODEL`, `EMBED_MODEL`, `FAISS_PATH`) установлены при необходимости.

In [7]:
# 2) Импорт библиотек и проверка локальных скриптов
import sys
from pathlib import Path
import importlib

print('python:', sys.version.splitlines()[0])

# Проверяем, что основная директория проекта доступна
repo_root = Path('..').resolve()
print('repo_root (relative):', repo_root)

# Files we expect (adjust names if your project uses different filenames)
expected = ['scripts/build_vector_db.py', '../app/main.py', '../app/model_selector.py', '../app/model_runner.py']
for p in expected:
    exists = Path(p).exists()
    print(f"{p}: {exists}")

# Quick import checks (non-fatal)
for mod in ['app.model_selector', 'app.model_runner', 'app.vector_db']:
    try:
        importlib.import_module(mod)
        print(f"Imported {mod}")
    except Exception as e:
        print(f"Failed to import {mod}: {e}")


python: 3.12.3 (main, Aug 14 2025, 17:47:21) [GCC 13.3.0]
repo_root (relative): /home/sasha/llm-ida
scripts/build_vector_db.py: False
../app/main.py: True
../app/model_selector.py: True
../app/model_runner.py: True
Imported app.model_selector
Imported app.model_runner
Imported app.vector_db


## 3) Генерация синтетического пространственно‑временного датасета

Создадим небольшой синтетический датасет: X с формой (T, N, F). T=50 временных шагов, N=8 локаций, F=1 признак (например, интенсивность). Добавим синусоиды с пространственным градиентом и шум.

In [2]:
import numpy as np
import json

# parameters
T = 50
N = 8
F = 1

x = np.linspace(0, 2 * np.pi, T)
X = np.zeros((T, N, F), dtype=np.float32)
for n in range(N):
    phase = n * 0.3
    X[:, n, 0] = np.sin(x + phase) + 0.1 * n + 0.2 * np.random.randn(T)

meta = {"nodes": list(range(N)), "freq": "synthetic"}
np.savez('demo_in.npz', X=X, meta=meta)
print('Saved demo_in.npz with X shape', X.shape)

# small JSON dump usable for small tests (not recommended for big arrays)
with open('demo_in.json', 'w', encoding='utf-8') as f:
    json.dump({'X': X[:10].tolist(), 'meta': meta}, f)
print('Saved demo_in.json (first 10 timesteps)')


Saved demo_in.npz with X shape (50, 8, 1)
Saved demo_in.json (first 10 timesteps)


## 4) Примеры задач и форматы входных данных

Форматы входных данных, которые поддерживаются/рекомендуются:

- NPZ/NPY: массивы numpy (рекомендуется для больших объёмов): `np.savez('in.npz', X=X, meta=meta)`
- CSV: табличный формат (time, node, feat1,...,target)
- JSON: для небольших payload'ов (вложенные списки)

Примеры задач:
- Одношаговое прогнозирование (h=1)
- Мультишаговое (h>1)
- Заполнение пропусков (imputation)

Пример JSON-payload (одно наблюдение):

```json
{
  "meta": {"nodes": [0,1,2]},
  "X": [[[0.1],[0.2]], [[0.15],[0.25]]]
}
```

## 5) Подготовка payload (JSON / NPZ) для локального запуска

Мы уже сохранили `demo_in.npz` и `demo_in.json`. Для снижения размера используйте `float32` и при необходимости сжимайте архив (например, `np.savez_compressed`).

Примеры сериализации данных в JSON (маленькие объёмы):

```python
with open('demo_in.json','w') as f:
    json.dump({'X': X[:10].tolist(), 'meta': meta}, f)
```

## 6) Проверка интерфейсов локальных скриптов (signature checks)

Попробуем импортировать примерную функцию `predict` из `app.model_runner` или скрипта `predict.py` (если он есть). Код ниже вызывает `run_model` из `app.model_runner` с небольшим фрагментом данных и ловит исключения.

In [9]:
from importlib import import_module
import io, csv

# Try to use the project's model_runner API
try:
    mr = import_module('app.model_runner')
    print('Found app.model_runner, run_model exists:', hasattr(mr, 'run_model'))
    # use a tiny slice
    sample = X[:5]
    # if sample is ndarray, convert to a simple CSV with columns (t,node,value)
    if hasattr(sample, 'shape'):
        buf = io.StringIO()
        writer = csv.writer(buf)
        writer.writerow(['t', 'node', 'value'])
        T_s = sample.shape[0]
        N_s = sample.shape[1]
        for t in range(T_s):
            for n in range(N_s):
                v = float(sample[t, n, 0])
                writer.writerow([t, n, v])
        csv_text = buf.getvalue()
    else:
        # assume it's already a CSV-like string
        csv_text = sample

    try:
        code, predictions = mr.run_model('sktime', csv_text)
        print('run_model returned code length:', len(code))
        print('predictions (first lines):')
        for l in predictions.splitlines()[:8]:
            print('  ', l)
    except Exception as e:
        print('run_model call failed (this may be OK if backend libs missing):', type(e).__name__, e)
except Exception as e:
    print('Could not import app.model_runner:', e)


Found app.model_runner, run_model exists: True
run_model returned code length: 200
predictions (first lines):
   horizon,forecast
   1,1.1157123848795891
   2,1.1157123848795891
   3,1.1157123848795891


## 7) Запуск скрипта предсказания через subprocess и сбор вывода

Ниже шаблон для запуска CLI-скрипта `predict.py` или любого скрипта, который реализует интерфейс `--input/--output`. Скрипт в проекте может называться иначе — отредактируйте путь при необходимости.

In [10]:
import subprocess

script = 'predict.py'  # change if your project's script has another name
if Path(script).exists():
    cmd = ['python3', script, '--input', 'demo_in.npz', '--output', 'demo_out.npz']
    print('Running:', ' '.join(cmd))
    try:
        r = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        print('returncode:', r.returncode)
        print('stdout:\n', r.stdout[:1000])
        print('stderr:\n', r.stderr[:1000])
    except subprocess.TimeoutExpired:
        print('Process timed out')
else:
    print(script, 'not found; skip subprocess run')


predict.py not found; skip subprocess run


## 8) Валидация формата выходных данных и вычисление метрик

Пример проверки выходного NPZ и вычисления RMSE/MAE.

In [5]:
import math
import numpy as np

def rmse(a, b):
    return math.sqrt(np.mean((a - b) ** 2))

def mae(a, b):
    return float(np.mean(np.abs(a - b)))

out_path = Path('demo_out.npz')
if out_path.exists():
    arr = np.load(out_path, allow_pickle=True)
    y_pred = arr.get('y_pred') if 'y_pred' in arr else None
    print('y_pred keys in archive:', list(arr.keys()))
    if y_pred is not None:
        print('y_pred shape:', y_pred.shape)
        # compare small slice if truth exists
        if 'X' in locals():
            truth = X[:y_pred.shape[0]]
            print('RMSE:', rmse(truth, y_pred))
else:
    print('demo_out.npz not found; skip validation')


demo_out.npz not found; skip validation


## 9) Визуализация входа/выхода по пространству и времени

Примеры: временной ряд для узла 0, тепловая карта ошибок.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# plot a time series for node 0
if 'X' in globals():
    plt.figure(figsize=(8,3))
    plt.plot(X[:,0,0], label='node0')
    plt.title('Node 0 timeseries')
    plt.legend()
    plt.show()

# if prediction present, heatmap errors over time and nodes
if out_path.exists() and 'y_pred' in locals() and y_pred is not None:
    # compute error matrix (T,N)
    err = np.mean((truth - y_pred) ** 2, axis=2) if truth.shape == y_pred.shape else None
    if err is not None:
        plt.figure(figsize=(8,4))
        sns.heatmap(err.T, cmap='magma')
        plt.title('MSE over time (rows=time, cols=node)')
        plt.show()
else:
    print('Prediction not available for heatmap')


## 10) Отладка: логи, режимы --debug и распространённые ошибки

- Используйте `--debug` и `LOG_LEVEL=DEBUG` если скрипт поддерживает.
- Частые ошибки: mismatch shapes, wrong dtype (float64 vs float32), отсутствие обязательных ключей в NPZ/JSON.
- Добавляйте assert-ы на границах функций, и логирование форм входов/выходов.

## 11) Запуск unit-тестов и использование pytest в VSCode

Запускайте отдельные тесты для predict:

```bash
pytest -q tests/test_predict.py::test_predict_small
```

В VSCode: используйте Test Explorer, настройте интерпретатор и запустите/отлаживайте тесты прямо из IDE.

## 12) Интеграция с VSCode: breakpoints и Output Pane

- Откройте `predict.py` или `app/model_runner.py` в редакторе, поставьте breakpoint и используйте `Run > Start Debugging`.
- В интегрированном терминале запускайте скрипты и смотрите Output Pane для stdout/stderr.
- Если используете `python -m pdb` — удобно для пошаговой отладки в терминале.

## 13) Сценарии восстановления: таймауты, сериализация, несоответствие форматов

- Если subprocess падает с таймаутом — увеличьте `timeout` и запустите локально в терминале для диагностики.
- При ошибках сериализации — проверьте dtype и используйте `np.savez_compressed` или уменьшите размер перед сохранением в JSON.
- Для несоответствия форматов используйте `np.transpose` и печать форм `print(arr.shape)` на границах.

In [ ]:
{
  "cell_type": "markdown",
  "metadata": {
    "language": "markdown"
  },
  "source": [
    "## TinyLlama smoke-test\n",
    "\n",
    "This test loads the `TinyLlama/TinyLlama_v1.1` model via the Hugging Face `transformers` pipeline and runs the `select_model` function from `app.llm`.\n",
    "It demonstrates: \n",
    "- activating the project's `.venv`,\n",
    "- setting the `HF_MODEL` environment variable to `TinyLlama/TinyLlama_v1.1`,\n",
    "- invoking `select_model(...)` with a sample dataset description, and\n",
    "- printing the model's JSON response (or the fallback outcome).\n",
    "\n",
    "Run this cell to reproduce the quick smoke-test used during development."
  ]
}


In [2]:
# TinyLlama smoke-test (direct Python)\n# This cell imports `app.llm` directly from the repo, sets HF_MODEL, and calls select_model.
import os
import sys
from pathlib import Path
# Ensure repo root is on sys.path (not required if package installed editable)
repo_root = Path('.').resolve()
if str(repo_root) not in sys.path:    
    sys.path.insert(0, str(repo_root))
# Set the HF_MODEL for this test
os.environ['HF_MODEL'] = 'TinyLlama/TinyLlama_v1.1'
# Import and run the select_model function
import importlib
m = importlib.import_module('app.llm')
importlib.reload(m)
print('HF_MODEL=', m.HF_MODEL)
res = m.select_model('short timeseries from sensor network', 'forecast', ['pysteps','sktime','tslearn','torch_geometric'])
print('select_model ->', res)

HF_MODEL= TinyLlama/TinyLlama_v1.1


Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


select_model -> {'model_choice': 'pysteps', 'library': 'pysteps', 'rationale': 'pysteps is the most accurate model for this dataset', 'confidence': 1}
